# Projeto de Regressão: Previsão de Preços do Airbnb no Rio de Janeiro

**Discente:** Júlia Alanne Silvino dos Santos

**Discente:** Pablo Durkheim Fernandes do Nascimento

# **Alterações:**
___________________________________________________________________
- ## As colunas latitude e longitude foram substituídas pelo novo atributo distancia_centro_turistico. Este atributo calcula a distância de cada ponto até o centro da praia de Copacabana, transformando coordenadas em uma variável de negócio mais relevante.

- ## O número de épocas foi estendido para 200, mantendo-se a mesma arquitetura de rede, otimizador e taxa de aprendizado para isolar o impacto das mudanças.

- ## A combinação dessas duas alterações resultou em uma melhora significativa no desempenho, reduzindo o RMSE para 175,91.

## 1. Introdução e Definição do Problema

O objetivo deste projeto é desenvolver um modelo de Machine Learning capaz de prever o preço de diárias de imóveis anunciados na plataforma Airbnb na cidade do Rio de Janeiro. Para isso, utilizaremos um dataset público do portal *Inside Airbnb*.

O desenvolvimento seguirá um pipeline padrão de projetos de ciência de dados:
*  **Análise Exploratória dos Dados (EDA):** Para entender a estrutura, a


qualidade e os padrões dos dados.
*  **Pré-Processamento e Limpeza:** Para preparar os dados para a modelagem.
*  **Modelagem:** Utilizaremos a biblioteca LazyPredict para criar um *baseline* de desempenho com diversos modelos tradicionais e, em seguida, desenvolveremos um modelo de rede neural com PyTorch para buscar um resultado otimizado.
*  **Avaliação e Conclusão:** Analisaremos os resultados para determinar a eficácia dos modelos.

## 2. Preparação do Ambiente e Carregamento dos Dados

Nesta primeira etapa, importamos todas as bibliotecas que serão utilizadas ao longo do projeto, como `pandas` para manipulação de dados, `matplotlib` e `seaborn` para visualização, e `pycaret` para a modelagem automatizada.

Em seguida, carregamos o nosso conjunto de dados, o arquivo `listings.csv`, que contém as informações detalhadas sobre os anúncios.

In [ ]:
# Instalação da biblioteca LazyPredict
# O PyTorch já vem pré-instalado na maioria dos ambientes Colab.
!pip install lazypredict

In [ ]:
# Importação das bibliotecas básicas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 3. Análise Exploratória (EDA), Limpeza e Seleção de Features
 Uma análise profunda e uma limpeza cuidadosa garantem que o modelo será treinado com dados de alta qualidade, o que impacta diretamente sua performance. Dessa forma, nosso objetivo é:
1.  Entender a fundo cada variável.
2.  Tratar inconsistências, como valores ausentes e formatos incorretos.
3.  Selecionar as features mais promissoras para prever o preço.



In [ ]:
# Configuração para exibir todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Carregamento dos dados a partir do link fornecido no notebook base
# O comando !wget baixa o arquivo e o !gunzip descompacta.
!wget -O listings.csv.gz "https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2025-03-19/data/listings.csv.gz"
!gunzip -k listings.csv.gz

# Leitura do arquivo CSV para um DataFrame do Pandas
df_raw = pd.read_csv("listings.csv")

# Visualização das primeiras linhas para entender os dados
print("Visualização inicial dos dados:")
display(df_raw.head())

# Verificação das informações gerais do DataFrame, como tipos de dados e valores nulos
print("\nInformações gerais do DataFrame:")
df_raw.info()

--2025-09-22 00:10:27--  https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2025-03-19/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 13.249.141.60, 13.249.141.88, 13.249.141.25, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|13.249.141.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21857120 (21M) [application/x-gzip]
Saving to: ‘listings.csv.gz’

listings.csv.gz     100%[===================>]  20.84M   114MB/s    in 0.2s    

2025-09-22 00:10:27 (114 MB/s) - ‘listings.csv.gz’ saved [21857120/21857120]

gzip: listings.csv already exists; do you wish to overwrite (y or n)? y
Visualização inicial dos dados:


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2302715,https://www.airbnb.com/rooms/2302715,20250319150537,2025-03-20,city scrape,guilherme,"House with five large bedrooms, living room, g...",NaN,https://a0.muscache.com/pictures/30842883/b19c...,11706874,https://www.airbnb.com/users/show/11706874,Guilherme,2014-01-27,"Rio, Brazil",NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/11706874/prof...,https://a0.muscache.com/im/users/11706874/prof...,NaN,2.00,2.00,"['email', 'phone']",t,f,NaN,Anil,NaN,-22.95,-43.33,Entire home,Entire home/apt,12,5.00,5 baths,5.00,10.00,"[""Washer"", ""Hot tub"", ""Breakfast"", ""TV with st...","$2,700.00",10,30,10,10,30,30,10.00,30.00,NaN,t,30,60,90,365,2025-03-20,0,0,0,287,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,2304688,https://www.airbnb.com/rooms/2304688,20250319150537,2025-03-20,city scrape,OLIMPÍADAS 2016,"Apartment with living room, 2 bedrooms and kit...","Beaches, restaurants, supermarkets, amusement ...",https://a0.muscache.com/pictures/30867360/fe14...,10405816,https://www.airbnb.com/users/show/10405816,Eurico,2013-12-03,"Rio de Janeiro, Brazil",NaN,NaN,NaN,0%,f,https://a0.muscache.com/im/users/10405816/prof...,https://a0.muscache.com/im/users/10405816/prof...,NaN,1.00,1.00,"['email', 'phone']",t,f,"Rio, Rio de Janeiro, Brazil",Ipanema,NaN,-22.99,-43.19,Entire rental unit,Entire home/apt,4,1.00,1 bath,2.00,3.00,"[""Washer"", ""TV with standard cable"", ""Wifi"", ""...",$837.00,7,14,7,7,14,14,7.00,14.00,NaN,t,30,60,90,365,2025-03-20,0,0,0,287,0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
2,2306547,https://www.airbnb.com/rooms/2306547,20250319150537,2025-03-20,city scrape,Amplo Apto com 3 Dormitórios em Copacabana - 1...,Excellent apartment in Copacabana. Cozy and la...,The Copacabana Neighborhood is independent an...,https://a0.muscache.com/pictures/4923e99d-7190...,4107346,https://www.airbnb.com/users/show/4107346,Vera Lucia,2012-11-09,"Rio de Janeiro, Brazil",Sou uma pessoa que adora receber turistas no R...,within a few hours,100%,62%,f,https://a0.muscache.com/im/users/4107346/profi...,https://a0.muscache.com/im/users/4107346/profi...,Copacabana,1.00,1.00,"['email', 'phone']",t,t,"Copacabana, Rio de Janeiro, Brazil",Copacabana,NaN,-22.97,-43.19,Entire rental unit,Entire home/apt,7,2.00,2 baths,3.00,6.00,"[""Dedicated workspace"", ""Dishes and silverware...",$800.00,5,90,5,5,90,90,5.00,90.00,NaN,t,25,47,77,257,2025-03-20,13,3,1,257,4,30,24000.00,2020-01-07,2025-03-05,5.00,5.00,4.92,5.00,4.92,5.00,4.77,NaN,f,1,1,0,0,0.21
3,2306767,https://www.airbnb.com/rooms/2306767,20250319150537,2025-03-20,city scrape,S


Informações gerais do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42013 entries, 0 to 42012
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            42013 non-null  int64  
 1   listing_url                                   42013 non-null  object 
 2   scrape_id                                     42013 non-null  int64  
 3   last_scraped                                  42013 non-null  object 
 4   source                                        42013 non-null  object 
 5   name                                          42013 non-null  object 
 6   description                                   40982 non-null  object 
 7   neighborhood_overview                         16819 non-null  object 
 8   picture_url                                   42013 non-null  object 
 9   host_id                    

### 3.1. Seleção de Features para o Modelo

A escolha das features é uma etapa fundamental para o sucesso do modelo. Selecionamos um conjunto de colunas que capturam os principais fatores que influenciam o preço de um aluguel, agrupados em quatro categorias principais:

* **Qualidade e Anfitrião:** Features como `host_is_superhost` e `review_scores_rating` servem como indicadores da qualidade do serviço e da confiabilidade do anfitrião.
* **Localização Geográfica:** As coordenadas `latitude` e `longitude` são essenciais, pois a localização é um dos fatores mais críticos na precificação de imóveis no Rio de Janeiro.
* **Características da Propriedade:** Atributos como `accommodates`, `bedrooms`, `property_type` e `room_type` descrevem o tamanho, a capacidade e a natureza do imóvel alugado.
* **Regras de Negócio e Popularidade:** Colunas como `minimum_nights` e `number_of_reviews` refletem a estratégia de negócio do anfitrião e a demanda pela propriedade.

Essa seleção multifacetada fornece ao modelo uma base de informações robusta para prever os preços com maior precisão.

In [ ]:
# Seleção de colunas que podem influenciar o preço do aluguel
# A escolha se baseia em características do imóvel e do anfitrião.
features_selecionadas = [
    'host_response_rate',       # Taxa de resposta do anfitrião
    'host_is_superhost',        # Se o anfitrião é "Superhost"
    'latitude',                 # Latitude (localização)
    'longitude',                # Longitude (localização)
    'property_type',            # Tipo da propriedade
    'room_type',                # Tipo do quarto (inteiro, privado, etc.)
    'accommodates',             # Número de hóspedes que acomoda
    'bathrooms_text',           # Descrição dos banheiros
    'bedrooms',                 # Número de quartos
    'beds',                     # Número de camas
    'price',                    # Preço (nossa variável alvo)
    'minimum_nights',           # Mínimo de noites
    'number_of_reviews',        # Número de avaliações
    'review_scores_rating',     # Nota média das avaliações
    'instant_bookable'          # Se permite reserva instantânea
]

df = df_raw[features_selecionadas].copy()

print("DataFrame após a seleção de features:")
display(df.head())

DataFrame após a seleção de features:


,host_response_rate,host_is_superhost,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable
0,NaN,f,-22.95,-43.33,Entire home,Entire home/apt,12,5 baths,5.00,10.00,"$2,700.00",10,0,NaN,f
1,NaN,f,-22.99,-43.19,Entire rental unit,Entire home/apt,4,1 bath,2.00,3.00,$837.00,7,0,NaN,f
2,100%,f,-22.97,-43.19,Entire rental unit,Entire home/apt,7,2 baths,3.00,6.00,$800.00,5,13,5.00,f
3,98%,f,-22.96,-43.18,Entire rental unit,Entire home/apt,3,1 bath,0.00,2.00,$190.00,3,193,4.69,f
4,100%,f,-22.97,-43.19,Entire rental unit,Entire home/apt,4,1 bath,1.00,2.00,$220.00,2,28,4.89,f


### 3.2. Tratamento, Limpeza e Pré-processamento dos Dados

Com base na análise exploratória, ficou claro que os dados brutos precisavam de um tratamento significativo antes de serem usados para a modelagem. Esta seção detalha o pipeline de limpeza e transformação que foi aplicado, passo a passo.

#### Etapa 1: Correção de Formato e Extração de Informação

O primeiro passo foi garantir que as colunas estivessem no formato numérico correto e que a informação útil fosse extraída de colunas textuais.

* **Tratamento da Variável Alvo (`price`):** A coluna `price` estava formatada como texto, contendo símbolos como `$` e `,`. Para usá-la em um modelo de regressão, removemos esses caracteres e a convertemos para um tipo numérico (float). Linhas em que o preço se tornou nulo após a conversão foram removidas, pois sem a variável alvo elas são inúteis para o treinamento.
* **Normalização da Taxa de Resposta (`host_response_rate`):** Esta coluna também era textual, com o símbolo `%`. Removemos o símbolo, convertemos para numérico e dividimos por 100 para que os valores representassem uma taxa entre 0 e 1, um formato padrão para modelos de machine learning.
* **Engenharia de Feature a partir de `bathrooms_text`:** A coluna de banheiros era um texto descritivo (ex: "1.5 baths", "Half-bath"). Usamos uma expressão regular para extrair apenas o valor numérico, criando uma nova coluna `bathrooms` puramente quantitativa. A coluna original (`bathrooms_text`) foi então descartada.

#### Etapa 2: Preenchimento de Valores Ausentes (Imputação)

Para evitar a perda de dados valiosos ao descartar linhas com informações faltantes, aplicamos estratégias de imputação:

* **Para Colunas Numéricas:** Nas colunas `bedrooms`, `beds`, `review_scores_rating`, `host_response_rate` e `bathrooms`, os valores ausentes foram preenchidos com a **mediana** de cada coluna. A mediana foi escolhida em vez da média por ser mais robusta a outliers (valores extremos).
* **Para Colunas Categóricas:** Na coluna `host_is_superhost`, os valores nulos foram preenchidos com a **moda** (o valor mais frequente). Essa abordagem preserva a distribuição original da categoria antes da sua conversão para formato numérico.

#### Etapa 3: Transformação Final para Formato Numérico

Finalmente, convertemos as colunas não-numéricas restantes para um formato que o modelo pudesse interpretar:

* **Conversão Booleana:** As colunas `host_is_superhost` e `instant_bookable`, que usavam os caracteres 't' (true) e 'f' (false), foram convertidas para um formato binário de **1 e 0**.
* **One-Hot Encoding:** As colunas categóricas `property_type` e `room_type` continham múltiplas categorias de texto. Aplicamos a técnica de *One-Hot Encoding*, que transforma cada categoria em uma nova coluna binária (0 ou 1). Isso permite que o modelo entenda essas categorias como features distintas, sem criar uma falsa relação de ordem entre elas.

In [ ]:
# --- Tratamento da variável alvo: 'price' ---
# A coluna 'price' está como texto (object) com '$' e ','. Precisamos convertê-la para número (float).
df['price'] = pd.to_numeric(df['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False), errors='coerce')

# Remove linhas onde o preço é nulo (passo essencial)
df.dropna(subset=['price'], inplace=True)

# Converte 'host_response_rate' para numérico
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str.replace('%', '', regex=False), errors='coerce') / 100.0

# Extrai o número de banheiros de 'bathrooms_text'
df['bathrooms'] = pd.to_numeric(df['bathrooms_text'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df.drop('bathrooms_text', axis=1, inplace=True)

# --- ETAPA 2: Preenchimento de Valores Nulos (Imputação) ---

# Colunas numéricas: preenche com a MEDIANA
cols_numericas = ['bedrooms', 'beds', 'review_scores_rating', 'host_response_rate', 'bathrooms']
for col in cols_numericas:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

# CORREÇÃO: Preenchemos os valores nulos de 'host_is_superhost' ANTES de converter. Usamos a moda (o valor mais comum).
mode_superhost = df['host_is_superhost'].mode()[0]
df['host_is_superhost'].fillna(mode_superhost, inplace=True)

# --- ETAPA 3: Transformação Final ---

# AGORA que não há mais nulos, convertemos 't'/'f' para 1/0.
df['host_is_superhost'] = df['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)
df['instant_bookable'] = df['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)

# One-Hot Encoding para as colunas categóricas restantes (sem 'drop_first=True' para clareza)
df = pd.get_dummies(df, columns=['property_type', 'room_type'])

# --- ETAPA 4: Verificação Final ---

print("--- Análise da Contagem de Linhas APÓS a Correção ---")
df.info()

print("\n--- DataFrame final após todas as transformações ---")
display(df.head())

--- Análise da Contagem de Linhas APÓS a Correção ---
<class 'pandas.core.frame.DataFrame'>
Index: 38356 entries, 0 to 42012
Data columns (total 95 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   host_response_rate                                38356 non-null  float64
 1   host_is_superhost                                 38356 non-null  int64  
 2   latitude                                          38356 non-null  float64
 3   longitude                                         38356 non-null  float64
 4   accommodates                                      38356 non-null  int64  
 5   bedrooms                                          38356 non-null  float64
 6   beds                                              38356 non-null  float64
 7   price                                             38356 non-null  float64
 8   minimum_nights                                 

,host_response_rate,host_is_superhost,latitude,longitude,accommodates,bedrooms,beds,price,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,bathrooms,property_type_Boat,property_type_Camper/RV,property_type_Campsite,property_type_Casa particular,property_type_Castle,property_type_Cave,property_type_Dome,property_type_Earthen home,property_type_Entire bungalow,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condo,property_type_Entire cottage,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Farm stay,property_type_Houseboat,property_type_Hut,property_type_Island,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in bungalow,property_type_Private room in casa particular,property_type_Private room in castle,property_type_Private room in chalet,property_type_Private room in condo,property_type_Private room in cottage,property_type_Private room in earthen home,property_type_Private room in farm stay,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in nature lodge,property_type_Private room in rental unit,property_type_Private room in resort,property_type_Private room in serviced apartment,property_type_Private room in shipping container,property_type_Private room in tent,property_type_Private room in tiny home,property_type_Private room in tower,property_type_Private room in townhouse,property_type_Private room in treehouse,property_type_Private room in vacation home,property_type_Private room in villa,property_type_Ranch,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in aparthotel,property_type_Shared room in bed and breakfast,property_type_Shared room in condo,property_type_Shared room in cottage,property_type_Shared room in earthen home,property_type_Shared room in guest suite,property_type_Shared room in guesthouse,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in hotel,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Shared room in townhouse,property_type_Shipping container,property_type_Tiny home,property_type_Treehouse,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,1.00,0,-22.95,-43.33,12,5.00,10.00,2700.00,10,0,4.92,0,5.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,1.00,0,-22.99,-43.19,4,2.00,3.00,837.00,7,0,4.92,0,1.00,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,Fals

TESTE

In [ ]:
from math import radians, sin, cos, sqrt, atan2

# Função para calcular a distância entre duas coordenadas (Fórmula de Haversine)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    a = sin(dLat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dLon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Coordenadas do Posto 4 de Copacabana (ponto central)
PONTO_CENTRAL_LAT, PONTO_CENTRAL_LON = -22.9714, -43.1825

# df é o seu dataframe após a limpeza inicial
df['distancia_centro_turistico'] = df.apply(
    lambda row: haversine_distance(PONTO_CENTRAL_LAT, PONTO_CENTRAL_LON, row['latitude'], row['longitude']),
    axis=1
)

# Agora, podemos remover as colunas originais, pois a distância é mais informativa
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

print("Nova feature 'distancia_centro_turistico' criada.")
display(df[['distancia_centro_turistico']].head())

Nova feature 'distancia_centro_turistico' criada.


,distancia_centro_turistico
0,15.19
1,1.96
2,0.66
3,0.99
4,0.45


### 3.2. Tratamento de Outliers com o Método IQR

Outliers são pontos de dados que se desviam significativamente do resto do conjunto. Em nosso contexto, um anúncio com um preço de R$50.000 por noite ou uma exigência de 500 noites mínimas são exemplos de outliers. Esses valores extremos podem distorcer o treinamento do modelo, fazendo com que ele aprenda padrões que não se aplicam à maioria dos dados, o que prejudica sua capacidade de generalização.

Para mitigar esse problema, utilizamos um método estatístico robusto chamado **Intervalo Interquartil (IQR)**.

**Como funciona o método IQR?**

1.  **Cálculo dos Quartis:** Primeiro, calculamos o primeiro quartil (Q1), que é o valor que separa os 25% menores dados, e o terceiro quartil (Q3), que separa os 75% menores dados.
2.  **Cálculo do IQR:** O IQR é a diferença entre Q3 e Q1. Ele representa a faixa onde se encontram os 50% "centrais" dos dados.
3.  **Definição dos Limites:** Criamos um "limite" superior e inferior usando a fórmula:
    * `Limite Inferior = Q1 - 1.5 * IQR`
    * `Limite Superior = Q3 + 1.5 * IQR`
4.  **Filtragem:** Qualquer ponto de dado que esteja fora desses limites é considerado um outlier e é removido do nosso dataset.

Aplicamos este método iterativamente para um conjunto de colunas numéricas críticas, como `price`, `accommodates`, `minimum_nights`, e as coordenadas geográficas, garantindo que nosso conjunto de dados de treinamento se tornasse mais denso e representativo da maioria dos anúncios do Airbnb.

**Resultado:**

Ao aplicar este filtro, conseguimos remover **14.131** linhas que continham valores extremos, resultando em um dataset mais limpo e robusto para a etapa de modelagem.

In [ ]:
# DataFrame 'df' deve ser aquele após a limpeza e transformação, mas ANTES da remoção de outliers.
# Certifique-se de ter executado o código de limpeza da resposta anterior.

# --- 1. Definir as colunas para verificar outliers ---
# Escolhemos colunas numéricas onde valores extremos não fazem sentido prático.
colunas_para_robustecer = [
    'distancia_centro_turistico',
    # 'latitude',                 # Latitude (localização) - Removido após criar distancia_centro_turistico
    # 'longitude',                # Longitude (localização) - Removido após criar distancia_centro_turistico
    'accommodates',             # Número de hóspedes que acomoda
    'bedrooms',                 # Número de quartos
    'beds',                     # Número de camas
    'price',                    # Preço (nossa variável alvo)
    'minimum_nights',           # Mínimo de noites
    'number_of_reviews',        # Número de avaliações
    'review_scores_rating',     # Nota média das avaliações

]

df_antes = df.copy()
print(f"Número de linhas antes da remoção robusta: {len(df_antes)}")

# --- 2. Aplicar o filtro IQR para cada coluna ---
df_depois = df_antes.copy()

for col in colunas_para_robustecer:
    Q1 = df_depois[col].quantile(0.25)
    Q3 = df_depois[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Filtra o DataFrame, mantendo apenas as linhas dentro dos limites para a coluna atual
    df_depois = df_depois[(df_depois[col] >= limite_inferior) & (df_depois[col] <= limite_superior)]

print(f"Número de linhas após a remoção robusta: {len(df_depois)}")
print(f"Total de linhas removidas: {len(df_antes) - len(df_depois)}")

# Agora você pode seguir com a separação de X e y a partir de 'df_depois'
X_clean_robusto = df_depois.drop('price', axis=1)
y_clean_robusto = df_depois['price']

Número de linhas antes da remoção robusta: 38356
Número de linhas após a remoção robusta: 25900
Total de linhas removidas: 12456


### 3.4. Normalização

#### Normalização dos Dados com MinMaxScaler

Modelos de Machine Learning, especialmente redes neurais e algoritmos baseados em distância, são sensíveis à escala das features. Uma feature com uma faixa de valores muito grande (como o `price`) pode acabar "dominando" outras features com faixas menores (como `review_scores_rating`), fazendo com que o modelo dê uma importância indevida a ela. A normalização coloca todas as features em uma mesma escala, garantindo que todas contribuam de forma equilibrada para o resultado.

Utilizamos o `MinMaxScaler`. Este método transforma cada feature de forma que todos os seus valores fiquem contidos no intervalo entre 0 e 1. A fórmula básica é:

*x_scaled = (x - min(x)) / (max(x) - min(x))*

Isso preserva a forma da distribuição original dos dados, mas ajusta sua escala, sendo uma excelente escolha para a maioria dos algoritmos que usaremos.

#### Divisão em Conjuntos de Treino e Teste

O objetivo final de um modelo de Machine Learning é fazer previsões precisas sobre dados novos. Para simular essa situação e avaliar a real capacidade de generalização do nosso modelo, dividimos nosso dataset em dois conjuntos:

1.  **Conjunto de Treino (80% dos dados):** É a porção dos dados que o modelo usará para aprender os padrões e ajustar seus parâmetros.
2.  **Conjunto de Teste (20% dos dados):** É mantido "escondido" durante o treinamento. Após o modelo ser treinado, usamos este conjunto para fazer previsões e comparar com os valores reais, obtendo uma métrica de desempenho imparcial.

A função `train_test_split` do Scikit-learn automatiza essa separação de forma aleatória. O parâmetro `random_state=42` é usado para garantir que a divisão seja sempre a mesma cada vez que o código for executado, o que torna nossos resultados reprodutíveis.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalização Min-Max
# Ajusta todas as features para o intervalo [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_clean_robusto)

# Divisão dos dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_clean_robusto.values, test_size=0.2, random_state=42)

print(f"Dimensões do conjunto de treino (X): {X_train.shape}")
print(f"Dimensões do conjunto de teste (X): {X_test.shape}")

Dimensões do conjunto de treino (X): (20720, 93)
Dimensões do conjunto de teste (X): (5180, 93)


## 4. Modelagem com PyTorch

Após a análise e o pré-processamento, desenvolvemos o modelo principal de regressão com PyTorch, uma biblioteca de Deep Learning que permite a criação de redes neurais flexíveis para aprender padrões complexos nos dados.

### 4.1. Preparação dos Dados
Para que o PyTorch possa processar os dados, primeiro convertemos os conjuntos de treino e teste do formato NumPy para **Tensores** do PyTorch. Em seguida, utilizamos `DataLoaders` para organizar os dados em lotes (*batches*), o que otimiza o treinamento e melhora a capacidade de generalização do modelo.

### 4.2. Definição da Arquitetura da Rede Neural
O "cérebro" do modelo foi definido na classe `Architecture`. Trata-se de uma rede neural simples composta por:
* **Camadas Lineares (`nn.Linear`):** Camadas de neurônios que realizam transformações nos dados.
* **Funções de Ativação (`nn.ReLU`):** Introduzem não-linearidade, permitindo que o modelo aprenda relações complexas.
* **Camada de Saída:** Uma camada final que produz um único valor: a previsão do preço.

### 4.3. Treinamento do Modelo
O treinamento é o processo de ensinar o modelo a minimizar seus erros de previsão.
* **Função de Perda (`MSELoss`):** Usamos o Erro Quadrático Médio para medir a diferença entre os preços previstos e os reais.
* **Otimizador (`Adam`):** Este algoritmo ajusta os pesos internos do modelo para reduzir a perda calculada.
* **Loop de Treinamento:** O modelo processa o conjunto de dados de treino por 50 ciclos (*épocas*), ajustando-se a cada passo para se tornar progressivamente mais preciso.

### 4.4. Avaliação do Desempenho
Após o treinamento, avaliamos a performance do modelo no conjunto de teste (dados nunca vistos antes) usando duas métricas principais:
* **RMSE (Root Mean Squared Error):** Indica o erro médio das previsões na mesma unidade da variável alvo (Reais), tornando o resultado facilmente interpretável.
* **R² Score:** Mostra a proporção da variação dos preços que o modelo consegue explicar. Um valor próximo de 1 indica um modelo de alta qualidade.

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# --- Fixando semente para reprodutibilidade ---
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# --- Normalização dos dados ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertendo para tensores
X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
y_train_tensor = torch.tensor(y_train.astype(np.float32)).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
y_test_tensor = torch.tensor(y_test.astype(np.float32)).view(-1, 1)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# --- Arquitetura da rede melhorada ---
class Architecture(nn.Module):
    def __init__(self, input_size):
        super(Architecture, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.drop1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.drop2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.drop3 = nn.Dropout(0.3)

        self.out = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.drop3(x)
        x = self.out(x)
        return x

# --- Instanciar modelo ---
input_dim = X_train.shape[1]
model_pytorch = Architecture(input_size=input_dim)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model_pytorch.parameters(), lr=0.001, weight_decay=1e-4)


# --- Treinamento com Early Stopping ---
print("\nIniciando o treinamento do modelo PyTorch...")
num_epochs = 200
patience = 20
best_loss = float("inf")
trigger_times = 0

for epoch in range(num_epochs):
    model_pytorch.train()
    for features, labels in train_loader:
        outputs = model_pytorch(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validação
    model_pytorch.eval()
    with torch.no_grad():
        val_outputs = model_pytorch(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor).item()


    if (epoch + 1) % 10 == 0:
        print(f"Época [{epoch+1}/{num_epochs}], Val_Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        trigger_times = 0
        best_model_state = model_pytorch.state_dict()
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping na época {epoch+1}")
            break

# Carregar melhor modelo salvo
model_pytorch.load_state_dict(best_model_state)

# --- Avaliação ---
model_pytorch.eval()
with torch.no_grad():
    y_pred_tensor = model_pytorch(X_test_tensor)
    y_pred = y_pred_tensor.numpy()

rmse_pytorch = np.sqrt(mean_squared_error(y_test, y_pred))
r2_pytorch = r2_score(y_test, y_pred)

print("\n--- Resultados do Modelo PyTorch ---")
print(f"RMSE: {rmse_pytorch:.2f}")
print(f"R² Score: {r2_pytorch:.4f}")


Iniciando o treinamento do modelo PyTorch...
Época [10/200], Val_Loss: 32356.0273
Época [20/200], Val_Loss: 32227.1016
Época [30/200], Val_Loss: 32044.8984
Época [40/200], Val_Loss: 31770.5664
Época [50/200], Val_Loss: 31062.9805
Época [60/200], Val_Loss: 30862.4473
Época [70/200], Val_Loss: 30690.8203
Época [80/200], Val_Loss: 30887.9004
Época [90/200], Val_Loss: 30716.0645
Época [100/200], Val_Loss: 30622.7930
Época [110/200], Val_Loss: 30556.1855
Época [120/200], Val_Loss: 30601.9863
Early stopping na época 126

--- Resultados do Modelo PyTorch ---
RMSE: 175.01
R² Score: 0.3463


## 5. Comparação e Baseline com LazyPredict

Para avaliar a eficácia do nosso modelo PyTorch, criamos um **ponto de referência (baseline)** usando a biblioteca `LazyPredict`.

O `LazyPredict` automatiza o treinamento e a avaliação de dezenas de modelos de Machine Learning tradicionais. Com um único comando, ele treina os modelos no mesmo conjunto de dados limpo e apresenta uma tabela classificatória, ranqueando os algoritmos por métricas de desempenho como R² e RMSE. O resultado nos mostra rapidamente quais modelos convencionais funcionam melhor para este problema, estabelecendo um benchmark para a nossa solução customizada.

In [ ]:
from lazypredict.Supervised import LazyRegressor

# O LazyRegressor não requer dados normalizados, mas vamos usá-los para uma comparação justa.
X_train_lp, X_test_lp, y_train_lp, y_test_lp = train_test_split(X_clean_robusto, y_clean_robusto, test_size=0.2, random_state=42)


# Inicializa e treina os modelos
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train_lp, X_test_lp, y_train_lp, y_test_lp)

print("\n--- Resultados do LazyPredict (Top 5 Modelos) ---")
display(models.head(5))

  0%|          | 0/42 [00:00<?, ?it/s]